# Import libraries

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 

# Load Data 

In [13]:
eda_df = pd.read_csv('./src/notebooks/data/wesad_eda_empatica.csv', sep=';')
bvp_df = pd.read_csv('./src/notebooks/data/wesad_bvp_empatica.csv', sep=';')
acc_df = pd.read_csv('./src/notebooks/data/wesad_acc_empatica.csv', sep=';')
temp_df = pd.read_csv('./src/notebooks/data/wesad_temp_empatica.csv', sep=';')
resp_df = pd.read_csv('./src/notebooks/data/wesad_resp_empatica.csv', sep=';')
emg_df = pd.read_csv('./src/notebooks/data/wesad_emg_empatica.csv', sep=';')
ecg_df = pd.read_csv('./src/notebooks/data/wesad_ecg_empatica.csv', sep=';')
label_df = pd.read_csv('./src/notebooks/data/wesad_label_empatica.csv', sep=';')

# Feature extraction

In [61]:
def non_overlapping_rolling_stats(data, window_size):
    mean_vals = []
    std_vals = []
    min_vals = []
    max_vals = []
    
    for start in range(0, len(data), window_size):
        window_data = data[start:start + window_size]
        if len(window_data) == window_size:
            mean_vals.append(window_data.mean())
            std_vals.append(window_data.std())
            min_vals.append(window_data.min())
            max_vals.append(window_data.max())
    
    result = pd.DataFrame({
        'mean': mean_vals,
        'std': std_vals,
        'min': min_vals,
        'max': max_vals
    })
    return result

In [59]:
window_size = 4
eda_features = non_overlapping_rolling_stats(eda_df[eda_df.columns[1]], window_size=window_size)
temp_features = non_overlapping_rolling_stats(temp_df[temp_df.columns[1]], window_size=window_size)

window_size = 32
x_acc_features = non_overlapping_rolling_stats(acc_df['x_axis'], window_size=window_size)
y_acc_features = non_overlapping_rolling_stats(acc_df['y_axis'], window_size=window_size)
z_acc_features = non_overlapping_rolling_stats(acc_df['z_axis'], window_size=window_size)

window_size = 64 
bvp_features = non_overlapping_rolling_stats(bvp_df['BVP'], window_size=window_size)

window_size = 700
resp_features = non_overlapping_rolling_stats(resp_df['RESP'], window_size=window_size)
emg_features = non_overlapping_rolling_stats(emg_df['EMG'], window_size=window_size)
ecg_features = non_overlapping_rolling_stats(ecg_df['ECG'], window_size=window_size)



In [81]:
import pandas as pd
import numpy as np
from scipy.stats import mode

window_size = 4

def non_overlapping_rolling_labels(data, window_size):
    mode_vals = []
    
    for start in range(0, len(data), window_size):
        window_data = data[start:start + window_size]
        if len(window_data) == window_size:
            mode_vals.append(mode(window_data).mode)  
    
    result = pd.DataFrame({
        'mode': mode_vals
    })
    return result

In [82]:
window_size = 700 
labels = non_overlapping_rolling_labels(label_df['label'], window_size=window_size)

In [90]:
# chech that the feature extraction is done correctly. 

print(f'----------------------------------------------')
print(f'labels shape : {labels.shape}')
print(f'eda_features shape : {eda_features.shape}')
print(f'temp_features shape : {temp_features.shape}')
print(f'x_acc_features shape : {x_acc_features.shape}')
print(f'y_acc_features shape : {y_acc_features.shape}')
print(f'z_acc_features shape : {z_acc_features.shape}')
print(f'resp_features shape : {resp_features.shape}')
print(f'emg_features shape : {emg_features.shape}')
print(f'ecg_features shape : {ecg_features.shape}')
print(f'labels shape : {labels.shape}')
print(f'----------------------------------------------')

labels shape : (82440, 1)
eda_features shape : (82440, 4)
temp_features shape : (82440, 4)
x_acc_features shape : (82440, 4)
y_acc_features shape : (82440, 4)
z_acc_features shape : (82440, 4)
resp_features shape : (82440, 4)
emg_features shape : (82440, 4)
ecg_features shape : (82440, 4)
labels shape : (82440, 1)
